In [1]:
!pip install arxiv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/81.3 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.3 kB 435.7 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/81.3 kB 281.8 kB/s eta 0:00:01
   ----------------------------------- ---- 71.7/81.3 kB 357.2 kB/s eta 0:00:01
   ---------------------------------------- 81.3/81.3 kB 380.2 kB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=1a1f897f057c11e305d642d05c5f8e2bab28de5208a8d2d91


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [15]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [16]:
wiki.name

'wikipedia'

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv("COHERE_API")
embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-english-v3.0",
    user_agent="MyApp/1.0" 
)

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

retriever_tool.name

'langsmith_search'

In [14]:
## arxiv tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [17]:
tools = [wiki, arxiv_tool, retriever_tool]

In [18]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025D20AC4CC0>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.

In [28]:
## agents

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent

In [29]:
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [30]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [35]:
## agent executer
from langchain.agents import AgentExecutor
agent_executor =AgentExecutor(agent=agent, tools = tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [36]:
agent_executor.invoke({"input": "tell me about langsmith"})



> Entering new AgentExecutor chain...
 I am a helpful assistant. Langsmith is a research project that focuses on developing a machine learning model for natural language understanding and generation. The project is led by researchers at the University of Oxford and aims to create a model that can understand and generate human-like text in a variety of languages. The project is part of the broader field of artificial intelligence (AI) and natural language processing (NLP). Langsmith's research is focused on developing models that can understand and generate text in a way that is more flexible and adaptable than current state-of-the-art models. The project is still ongoing and new research is being published regularly. For more information, you can visit the Langsmith website or check out the latest research papers on arXiv.org.

> Finished chain.


{'input': 'tell me about langsmith',
 'output': " I am a helpful assistant. Langsmith is a research project that focuses on developing a machine learning model for natural language understanding and generation. The project is led by researchers at the University of Oxford and aims to create a model that can understand and generate human-like text in a variety of languages. The project is part of the broader field of artificial intelligence (AI) and natural language processing (NLP). Langsmith's research is focused on developing models that can understand and generate text in a way that is more flexible and adaptable than current state-of-the-art models. The project is still ongoing and new research is being published regularly. For more information, you can visit the Langsmith website or check out the latest research papers on arXiv.org."}

In [37]:
agent_executor.invoke({"input": "tell me about machine learning with example"})



> Entering new AgentExecutor chain...
 Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.

Here's a simple example of machine learning:

Let's say we have a dataset of emails, some of which are spam and some of which are not. We want to create a machine learning model that can classify new emails as either spam or not spam.

1. First, we would label the emails in our dataset as either spam or not spam. This is known as training the model.

2. Next, we would feed the labeled data into a machine learning algorithm, such as a decision tree or a neural network. The algorithm will learn patterns in the data, such as certain words or phrases that are more common in spam emails.

3. Once the model has been trained, we can use it to classify new emails. For exa

{'input': 'tell me about machine learning with example',
 'output': ' Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.\n\nHere\'s a simple example of machine learning:\n\nLet\'s say we have a dataset of emails, some of which are spam and some of which are not. We want to create a machine learning model that can classify new emails as either spam or not spam.\n\n1. First, we would label the emails in our dataset as either spam or not spam. This is known as training the model.\n\n2. Next, we would feed the labeled data into a machine learning algorithm, such as a decision tree or a neural network. The algorithm will learn patterns in the data, such as certain words or phrases that are more common in spam emails.\n\n3. Once the model has been trained, we c

In [38]:
agent_executor.invoke({"input": "what's the paper 1605.08386 about"})



> Entering new AgentExecutor chain...
 The paper with the arXiv ID 1605.08386 is titled "Attention is All You Need". It is a seminal work in the field of Natural Language Processing (NLP) that introduced the Transformer model, a type of deep learning model that uses self-attention mechanisms to process sequences of data. The paper was authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper was published in 2017 and has had a significant impact on the development of NLP models.

> Finished chain.


{'input': "what's the paper 1605.08386 about",
 'output': ' The paper with the arXiv ID 1605.08386 is titled "Attention is All You Need". It is a seminal work in the field of Natural Language Processing (NLP) that introduced the Transformer model, a type of deep learning model that uses self-attention mechanisms to process sequences of data. The paper was authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper was published in 2017 and has had a significant impact on the development of NLP models.'}